In [1]:
!jupyter kernelspec list

Available kernels:
  python3    /usr/local/share/jupyter/kernels/python3


In [2]:
!cd /notebooks && source venv/bin/activate && ipython kernel install --user --name=venv
!pip -V

Installed kernelspec venv in /root/.local/share/jupyter/kernels/venv
pip 22.3.1 from /usr/local/lib/python3.9/dist-packages/pip (python 3.9)


In [1]:
from data_handling import get_data
import numpy as np
import torch.optim as optim

from models import *
from torch import nn
from best_params import best_params_dict

from torch_geometric.utils import to_dense_adj
from torch.nn import Sequential, ReLU, Linear


In [2]:
# Import the W&B Python Library and log into W&B
import wandb
wandb.login()

from run_GNN import *

# 1: Define objective/training function
def objective(config):
    opt = config
    best_val_acc_mean = 0
    for split in range(10):
        data = get_data(opt['dataset'],split)

        best_eval = 10000
        bad_counter = 0
        best_val_acc = 0

        if opt['model'] == 'GraphCON_GCN':
            model = GraphCON_GCN(nfeat=data.num_features,nhid=opt['nhid'],nclass=5,
                                 dropout=opt['drop'],nlayers=opt['nlayers'],dt=1.,
                                 alpha=opt['alpha'],gamma=opt['gamma'],res_version=opt['res_version']).to(opt['device'])
        elif opt['model'] == 'GraphCON_GAT':
            model = GraphCON_GAT(nfeat=data.num_features, nhid=opt['nhid'], nclass=5,
                                 dropout=opt['drop'], nlayers=opt['nlayers'], dt=1.,
                                 alpha=opt['alpha'], gamma=opt['gamma'],nheads=opt['nheads']).to(opt['device'])
        elif opt['model'] == "GraphCON_GRAFF":

            adj = to_dense_adj(data.edge_index)[0].to(opt['device'])
            model = GraphCON_GRAFF(nfeat=data.num_features,nhid=opt['nhid'],nclass=5,
                                 dropout=opt['drop'],nlayers=opt['nlayers'],dt=1.,
                                 alpha=opt['alpha'],gamma=opt['gamma'], A=adj, step_size=3 / opt["nlayers"], device=opt['device']).to(opt['device'])

        elif opt["model"] == "GCN":
            model = GCN(nfeat=data.num_features, nhid=opt['nhid'], nclass=5,
                                 dropout=opt['drop'], nlayers=opt['nlayers']).to(opt['device'])
        elif opt["model"] == "GAT":
            model = GAT(nfeat=data.num_features, nhid=opt['nhid'], nclass=5,
                                 dropout=opt['drop'], nlayers=opt['nlayers'], nheads=opt["nheads"]).to(opt['device'])
        elif opt["model"] == "GRAFF":
            adj = to_dense_adj(data.edge_index)[0].to(opt['device'])
            model = GRAFF(nfeat=data.num_features, nhid=opt['nhid'], nclass=5,
                                 dropout=opt['drop'], nlayers=opt['nlayers'], A=adj, step_size=3 / opt["nlayers"], device=opt["device"]).to(opt['device'])

        elif opt["model"] == "H2GCN":

            adj = to_dense_adj(data.edge_index)[0].to(opt['device']).to_sparse()
            model = H2GCN(nfeat=data.num_features, nhid=opt['nhid'], nclass=5,
                                 dropout=opt['drop'], nlayers=opt['nlayers'], adj = adj).to(opt['device'])
        elif opt["model"] == "GraphCON_H2GCN":
            adj = to_dense_adj(data.edge_index)[0].to(opt['device']).to_sparse()
            model = GraphCON_H2GCN(nfeat=data.num_features, nhid=opt['nhid'], nclass=5,
                                 dropout=opt['drop'], nlayers=opt['nlayers'], adj = adj,
                                   alpha=opt['alpha'], gamma=opt['gamma']).to(opt['device'])
        elif opt["model"] == "MLP":
            model = MLP(n_feat=data.num_features, n_hid=opt['nhid'], nclass=5, 
                                 dropout=opt['drop'], nlayers=opt["nlayers"]).to(opt['device'])
        else:
            raise Exception("Not a valid model specified")

        optimizer = optim.Adam(model.parameters(),lr=opt['lr'],weight_decay=opt['weight_decay'])
        lf = nn.CrossEntropyLoss()

        @torch.no_grad()
        def test(model, data):
            model.eval()
            logits, accs, losses = model(data), [], []
            for _, mask in data('train_mask', 'val_mask', 'test_mask'):
                loss = lf(out[mask], data.y.squeeze()[mask])
                pred = logits[mask].max(1)[1]
                acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
                accs.append(acc)
                losses.append(loss.item())
            return accs, losses

        for epoch in range(opt['epochs']):
            model.train()
            optimizer.zero_grad()
            out = model(data.to(opt['device']))
            loss = lf(out[data.train_mask], data.y.squeeze()[data.train_mask])
            loss.backward()
            optimizer.step()

            [train_acc, val_acc, test_acc], [train_loss, val_loss, test_loss] = test(model,data)

            if (val_loss < best_eval):
                best_eval = val_loss
                best_val_acc = val_acc
            else:
                bad_counter += 1

            if(bad_counter==opt['patience']):
                break

            log = 'Split: {:01d}, Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
            #print(log.format(split, epoch, train_acc, val_acc, test_acc))
            
        best_val_acc_mean += best_val_acc / 10

    return best_val_acc_mean

def main():
    wandb.init(project='GDL-mini-project')
    score = objective(wandb.config)
    wandb.log({'score': score})


"""
# 2: Define the search space
sweep_configuration = {
    'method': 'random',
    'metric': {'goal': 'maximize', 'name': 'score'},
    'parameters': 
    {
        'model':{'values':["H2GCN"]},
        'dataset': {'values':["wisconsin"]},
        'epochs': {'values':[1500]},
        'patience': {'values':[100]},
        'nlayers':{'values':[2,3,4,5,6,7,8,9,10]},
        'drop':{'min':0.,'max':0.5},
        #'alpha':{'min':0., 'max':2.},
        #'gamma':{'min':0., 'max':2.},
        #'alpha':{'values':[0]},
        'nhid':{'values':[64]},
        #'nheads':{'values':[4]},
        'lr':{'distribution':'log_uniform_values',  'min':1e-4, 'max':1e-1,},
        'weight_decay':{'distribution':'log_uniform_values',  'min':1e-7, 'max':1e-3,},
        'device':{'values':torch.device('cuda' if torch.cuda.is_available() else 'cpu')}
     }
}


# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project='H2GCN-sweep-wisconsin')
wandb.agent(sweep_id, function=main, count=50)
"""

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


'\n# 2: Define the search space\nsweep_configuration = {\n    \'method\': \'random\',\n    \'metric\': {\'goal\': \'maximize\', \'name\': \'score\'},\n    \'parameters\': \n    {\n        \'model\':{\'values\':["H2GCN"]},\n        \'dataset\': {\'values\':["wisconsin"]},\n        \'epochs\': {\'values\':[1500]},\n        \'patience\': {\'values\':[100]},\n        \'nlayers\':{\'values\':[2,3,4,5,6,7,8,9,10]},\n        \'drop\':{\'min\':0.,\'max\':0.5},\n        #\'alpha\':{\'min\':0., \'max\':2.},\n        #\'gamma\':{\'min\':0., \'max\':2.},\n        #\'alpha\':{\'values\':[0]},\n        \'nhid\':{\'values\':[64]},\n        #\'nheads\':{\'values\':[4]},\n        \'lr\':{\'distribution\':\'log_uniform_values\',  \'min\':1e-4, \'max\':1e-1,},\n        \'weight_decay\':{\'distribution\':\'log_uniform_values\',  \'min\':1e-7, \'max\':1e-3,},\n        \'device\':{\'values\':torch.device(\'cuda\' if torch.cuda.is_available() else \'cpu\')}\n     }\n}\n\n\n# 3: Start the sweep\nsweep_id = wa

In [3]:
# 2: Define the search space
sweep_configuration = {
    'method': 'random',
    'metric': {'goal': 'maximize', 'name': 'score'},
    'parameters': 
    {
        'model':{'values':["GCN"]},
        'dataset': {'values':["texas"]},
        'epochs': {'values':[1500]},
        'patience': {'values':[100]},
        'nlayers':{'values':[1,2,3,4,5,6,7,8,9,10]},
        'drop':{'min':0.,'max':0.5},
        #'alpha':{'min':0., 'max':2.},
        #'gamma':{'min':0., 'max':2.},
        #'alpha':{'values':[0]},
        'nhid':{'values':[64]},
        'nheads':{'values':[4]},
        'lr':{'distribution':'log_uniform_values',  'min':1e-4, 'max':1e-1,},
        'weight_decay':{'distribution':'log_uniform_values',  'min':1e-7, 'max':1e-3,},
        'device':{'values':torch.device('cuda' if torch.cuda.is_available() else 'cpu')}
     }
}


# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project='GCN-sweep-texas')
wandb.agent(sweep_id, function=main, count=50)

Create sweep with ID: jc0umofu
Sweep URL: https://wandb.ai/1077407/GCN-sweep-texas/sweeps/jc0umofu


wandb: Agent Starting Run: 54uob28l with config:
wandb: 	dataset: texas
wandb: 	device: cuda
wandb: 	drop: 0.07776022168201491
wandb: 	epochs: 1500
wandb: 	lr: 0.016342155706879816
wandb: 	model: GCN
wandb: 	nheads: 4
wandb: 	nhid: 64
wandb: 	nlayers: 8
wandb: 	patience: 100
wandb: 	weight_decay: 0.00017318635868602812
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 1077407. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.56271


wandb: Agent Starting Run: j8grbgzu with config:
wandb: 	dataset: texas
wandb: 	device: cuda
wandb: 	drop: 0.104816767306868
wandb: 	epochs: 1500
wandb: 	lr: 0.011533603283668918
wandb: 	model: GCN
wandb: 	nheads: 4
wandb: 	nhid: 64
wandb: 	nlayers: 4
wandb: 	patience: 100
wandb: 	weight_decay: 7.948234080340014e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.64237


...

score,▁
score,0.54746


wandb: Agent Starting Run: 67qwmkvv with config:
wandb: 	dataset: texas
wandb: 	device: cuda
wandb: 	drop: 0.4389059706971141
wandb: 	epochs: 1500
wandb: 	lr: 0.0005119906032062745
wandb: 	model: GCN
wandb: 	nheads: 4
wandb: 	nhid: 64
wandb: 	nlayers: 9
wandb: 	patience: 100
wandb: 	weight_decay: 3.509513761325063e-07
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.47966


wandb: Agent Starting Run: dowosc0d with config:
wandb: 	dataset: texas
wandb: 	device: cuda
wandb: 	drop: 0.2889143271186528
wandb: 	epochs: 1500
wandb: 	lr: 0.004002742010878032
wandb: 	model: GCN
wandb: 	nheads: 4
wandb: 	nhid: 64
wandb: 	nlayers: 3
wandb: 	patience: 100
wandb: 	weight_decay: 0.00010000435893293794
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.64576


In [4]:
# 2: Define the search space
sweep_configuration = {
    'method': 'random',
    'metric': {'goal': 'maximize', 'name': 'score'},
    'parameters': 
    {
        'model':{'values':["GCN"]},
        'dataset': {'values':["wisconsin"]},
        'epochs': {'values':[1500]},
        'patience': {'values':[100]},
        'nlayers':{'values':[1,2,3,4,5,6,7,8,9,10]},
        'drop':{'min':0.,'max':0.5},
        #'alpha':{'min':0., 'max':2.},
        #'gamma':{'min':0., 'max':2.},
        #'alpha':{'values':[0]},
        'nhid':{'values':[64]},
        #'nheads':{'values':[4]},
        'lr':{'distribution':'log_uniform_values',  'min':1e-4, 'max':1e-1,},
        'weight_decay':{'distribution':'log_uniform_values',  'min':1e-7, 'max':1e-3,},
        'device':{'values':torch.device('cuda' if torch.cuda.is_available() else 'cpu')}
     }
}


# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project='GCN-sweep-texas')
wandb.agent(sweep_id, function=main, count=50)

Create sweep with ID: ed077lx2
Sweep URL: https://wandb.ai/1077407/GCN-sweep-texas/sweeps/ed077lx2


wandb: Agent Starting Run: f3flhlki with config:
wandb: 	dataset: wisconsin
wandb: 	device: cuda
wandb: 	drop: 0.005938160380345614
wandb: 	epochs: 1500
wandb: 	lr: 0.0010076046144787106
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 3
wandb: 	patience: 100
wandb: 	weight_decay: 0.0003619898230414424
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.68875


wandb: Agent Starting Run: 5zrsp3da with config:
wandb: 	dataset: wisconsin
wandb: 	device: cuda
wandb: 	drop: 0.3678874391874253
wandb: 	epochs: 1500
wandb: 	lr: 0.0006173844295430858
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 8
wandb: 	patience: 100
wandb: 	weight_decay: 0.000760085540936027
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.52375


...

score,▁
score,0.615


wandb: Agent Starting Run: 150b1wua with config:
wandb: 	dataset: wisconsin
wandb: 	device: cuda
wandb: 	drop: 0.11509588432808228
wandb: 	epochs: 1500
wandb: 	lr: 0.015222774309848195
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 2
wandb: 	patience: 100
wandb: 	weight_decay: 1.78050386077781e-07
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.78125


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 1j0muko7 with config:
wandb: 	dataset: wisconsin
wandb: 	device: cuda
wandb: 	drop: 0.39906259017882095
wandb: 	epochs: 1500
wandb: 	lr: 0.007105447195512715
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 6
wandb: 	patience: 100
wandb: 	weight_decay: 8.37340686196946e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.61125


In [5]:
# 2: Define the search space
sweep_configuration = {
    'method': 'random',
    'metric': {'goal': 'maximize', 'name': 'score'},
    'parameters': 
    {
        'model':{'values':["GCN"]},
        'dataset': {'values':["cornell"]},
        'epochs': {'values':[1500]},
        'patience': {'values':[100]},
        'nlayers':{'values':[1,2,3,4,5,6,7,8,9,10]},
        'drop':{'min':0.,'max':0.5},
        #'alpha':{'min':0., 'max':2.},
        #'gamma':{'min':0., 'max':2.},
        #'alpha':{'values':[0]},
        'nhid':{'values':[64]},
        #'nheads':{'values':[4]},
        'lr':{'distribution':'log_uniform_values',  'min':1e-4, 'max':1e-1,},
        'weight_decay':{'distribution':'log_uniform_values',  'min':1e-7, 'max':1e-3,},
        'device':{'values':torch.device('cuda' if torch.cuda.is_available() else 'cpu')}
     }
}


# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project='GCN-sweep-cornell-2')
wandb.agent(sweep_id, function=main, count=50)

Create sweep with ID: v5xpk3aq
Sweep URL: https://wandb.ai/1077407/GCN-sweep-cornell-2/sweeps/v5xpk3aq


wandb: Agent Starting Run: e16h8zxd with config:
wandb: 	dataset: cornell
wandb: 	device: cuda
wandb: 	drop: 0.42480659641157315
wandb: 	epochs: 1500
wandb: 	lr: 0.005771278865917323
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 7
wandb: 	patience: 100
wandb: 	weight_decay: 3.0254608721928145e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.46949


wandb: Agent Starting Run: auu4hsv7 with config:
wandb: 	dataset: cornell
wandb: 	device: cuda
wandb: 	drop: 0.1550218336892873
wandb: 	epochs: 1500
wandb: 	lr: 0.0228159487387041
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 5
wandb: 	patience: 100
wandb: 	weight_decay: 1.553374459856611e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.53729


...

score,▁
score,0.48136


wandb: Agent Starting Run: b8tnakqe with config:
wandb: 	dataset: cornell
wandb: 	device: cuda
wandb: 	drop: 0.2386079998281848
wandb: 	epochs: 1500
wandb: 	lr: 0.001239801597093691
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 9
wandb: 	patience: 100
wandb: 	weight_decay: 5.835039999911949e-07
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.48136


wandb: Agent Starting Run: 107stgli with config:
wandb: 	dataset: cornell
wandb: 	device: cuda
wandb: 	drop: 0.4194375307558598
wandb: 	epochs: 1500
wandb: 	lr: 0.007903849298164805
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 7
wandb: 	patience: 100
wandb: 	weight_decay: 2.290522307397757e-07
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.47458


In [7]:
# 2: Define the search space
sweep_configuration = {
    'method': 'random',
    'metric': {'goal': 'maximize', 'name': 'score'},
    'parameters': 
    {
        'model':{'values':["GCN"]},
        'dataset': {'values':["cornell"]},
        'epochs': {'values':[1500]},
        'patience': {'values':[100]},
        'nlayers':{'values':[2,3,4,5,6,7,8,9,10]},
        'drop':{'min':0.,'max':0.5},
        #'alpha':{'min':0., 'max':2.},
        #'gamma':{'min':0., 'max':2.},
        #'alpha':{'values':[0]},
        'nhid':{'values':[64]},
        #'nheads':{'values':[4]},
        'lr':{'distribution':'log_uniform_values',  'min':1e-4, 'max':1e-1,},
        'weight_decay':{'distribution':'log_uniform_values',  'min':1e-7, 'max':1e-3,},
        'device':{'values':torch.device('cuda' if torch.cuda.is_available() else 'cpu')}
     }
}


# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project='GCN-sweep-cornell')
wandb.agent(sweep_id, function=main, count=50)

Create sweep with ID: vvba9guy
Sweep URL: https://wandb.ai/1077407/GCN-sweep-cornell/sweeps/vvba9guy


wandb: Agent Starting Run: ffw33hj9 with config:
wandb: 	dataset: cornell
wandb: 	device: cuda
wandb: 	drop: 0.025057070122283753
wandb: 	epochs: 1500
wandb: 	lr: 0.02620891598360218
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 7
wandb: 	patience: 100
wandb: 	weight_decay: 6.652673766083207e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.55424


wandb: Agent Starting Run: ogev1qdf with config:
wandb: 	dataset: cornell
wandb: 	device: cuda
wandb: 	drop: 0.42485240542384256
wandb: 	epochs: 1500
wandb: 	lr: 0.001698701915454668
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 6
wandb: 	patience: 100
wandb: 	weight_decay: 0.00011927887440637888
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.48814


...

score,▁
score,0.51695


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: cj984qka with config:
wandb: 	dataset: cornell
wandb: 	device: cuda
wandb: 	drop: 0.10972297225058252
wandb: 	epochs: 1500
wandb: 	lr: 0.04310813476916207
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 4
wandb: 	patience: 100
wandb: 	weight_decay: 2.9867245892751475e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.54237


wandb: Agent Starting Run: 2wt4ifuc with config:
wandb: 	dataset: cornell
wandb: 	device: cuda
wandb: 	drop: 0.09570690007269368
wandb: 	epochs: 1500
wandb: 	lr: 0.007243396074911428
wandb: 	model: GCN
wandb: 	nhid: 64
wandb: 	nlayers: 7
wandb: 	patience: 100
wandb: 	weight_decay: 0.0003056133386967165
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.57627


In [8]:
# 2: Define the search space
sweep_configuration = {
    'method': 'random',
    'metric': {'goal': 'maximize', 'name': 'score'},
    'parameters': 
    {
        'model':{'values':["GraphCON_GAT"]},
        'dataset': {'values':["wisconsin"]},
        'epochs': {'values':[1500]},
        'patience': {'values':[500]},
        'nlayers':{'values':[2,3,4,5,6,7,8,9,10]},
        'drop':{'min':0.,'max':0.5},
        'alpha':{'min':0., 'max':2.},
        'gamma':{'min':0., 'max':2.},
        #'alpha':{'values':[0]},
        'nhid':{'values':[64]},
        'nheads':{'values':[4]},
        'lr':{'distribution':'log_uniform_values',  'min':1e-4, 'max':1e-1,},
        'weight_decay':{'distribution':'log_uniform_values',  'min':1e-7, 'max':1e-3,},
        'device':{'values':torch.device('cuda' if torch.cuda.is_available() else 'cpu')}
     }
}


# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project='GraphCON_GAT-sweep-cornell')
wandb.agent(sweep_id, function=main, count=50)

Create sweep with ID: tzgqeuic
Sweep URL: https://wandb.ai/1077407/GraphCON_GAT-sweep-cornell/sweeps/tzgqeuic


wandb: Agent Starting Run: 43p9ad5l with config:
wandb: 	alpha: 0.25465034506743045
wandb: 	dataset: wisconsin
wandb: 	device: cuda
wandb: 	drop: 0.4897759607118111
wandb: 	epochs: 1500
wandb: 	gamma: 0.42121085249409096
wandb: 	lr: 0.0003439875717029566
wandb: 	model: GraphCON_GAT
wandb: 	nheads: 4
wandb: 	nhid: 64
wandb: 	nlayers: 4
wandb: 	patience: 500
wandb: 	weight_decay: 5.213164270795307e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.54875


wandb: Agent Starting Run: fg51wmpt with config:
wandb: 	alpha: 0.8650477099625635
wandb: 	dataset: wisconsin
wandb: 	device: cuda
wandb: 	drop: 0.04984992201331179
wandb: 	epochs: 1500
wandb: 	gamma: 0.21158169991076736
wandb: 	lr: 0.00028015423295863
wandb: 	model: GraphCON_GAT
wandb: 	nheads: 4
wandb: 	nhid: 64
wandb: 	nlayers: 10
wandb: 	patience: 500
wandb: 	weight_decay: 3.113961298955392e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.51375


...

score,▁
score,0.47625


wandb: Agent Starting Run: a2roycp8 with config:
wandb: 	alpha: 1.0431594514389175
wandb: 	dataset: wisconsin
wandb: 	device: cuda
wandb: 	drop: 0.09652806557889264
wandb: 	epochs: 1500
wandb: 	gamma: 0.48739862375413945
wandb: 	lr: 0.00992445076773233
wandb: 	model: GraphCON_GAT
wandb: 	nheads: 4
wandb: 	nhid: 64
wandb: 	nlayers: 10
wandb: 	patience: 500
wandb: 	weight_decay: 0.00017535978449290617
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.445


wandb: Agent Starting Run: vwqlidcr with config:
wandb: 	alpha: 0.20375351709953615
wandb: 	dataset: wisconsin
wandb: 	device: cuda
wandb: 	drop: 0.22702083541309492
wandb: 	epochs: 1500
wandb: 	gamma: 0.8324931325803251
wandb: 	lr: 0.0033118887799235244
wandb: 	model: GraphCON_GAT
wandb: 	nheads: 4
wandb: 	nhid: 64
wandb: 	nlayers: 6
wandb: 	patience: 500
wandb: 	weight_decay: 0.0006527227862771033
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


score,▁
score,0.57375
